In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Preprocessing of Image

In [2]:
def process_image(image):
    
    #Denoising the Image - Median Filter
    denoised_image = cv2.medianBlur(image, 5)
    
    #Blurring the Image - Gaussian Filter
    blurred_image = cv2.GaussianBlur(denoised_image, (5, 5), 0)

    #Applying colour space transformation
    hls_image=cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    hue=hls_image[:,:,0]
    lightness=hls_image[:,:,1]
    saturation=hls_image[:,:,2]

    #Calculating Sobel Gradient along x-axis on Lightness Channel
    sobelx = cv2.Sobel(lightness,cv2.CV_64F,1,0,ksize=5)
    sobelx_abs=np.abs(sobelx)
    #Scaling the gradient values
    sobelx_scaled=((sobelx_abs/np.max(sobelx_abs))*255).astype(np.uint8)

    #Thresholding gradient values on Lightness Channel
    sobelx_threshold=np.copy(sobelx_scaled)
    sobelx_threshold[(sobelx_scaled >= 50) & (sobelx_scaled<=255)]=1
    sobelx_threshold[sobelx_scaled < 50]=0

    #Thresholding Saturation Channel
    saturation_threshold=np.copy(saturation)
    saturation[(sobelx_scaled >= 150) & (sobelx_scaled<=255)]=1
    saturation[sobelx_scaled < 150]=0

    #Combining the Results
    combined_image=np.zeros(sobelx_threshold.shape)
    combined_image[((sobelx_threshold == 1) | (saturation_threshold == 1))]=1

    return combined_image

## Region of Interest

In [3]:
def region_of_interest(image):
    triangle_height=image.shape[0]
    roi_vertices = np.array([[(250, triangle_height), (1170, triangle_height), (600, 370)]], dtype=np.int32)
    mask = np.zeros_like(image, dtype=np.uint8)
    cv2.fillPoly(mask, roi_vertices, 255)
    masked_image=cv2.bitwise_and(image,mask)
    return masked_image

## Finding Equations of Left and Right Lanes

In [4]:
def classify_lines_by_slope(lines):
    left_lines = []
    right_lines = []

    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1) if x2 != x1 else 0
            if slope < 0:
                left_lines.append((slope, x1, y1, x2, y2))
            else:
                right_lines.append((slope, x1, y1, x2, y2))
    
    return left_lines, right_lines

def average_lines(lines):
    if len(lines) == 0:
        return None
    slopes = [line[0] for line in lines]
    x1 = np.mean([line[1] for line in lines])
    y1 = np.mean([line[2] for line in lines])
    x2 = np.mean([line[3] for line in lines])
    y2 = np.mean([line[4] for line in lines])

    return x1, y1, x2, y2

In [5]:
def extrapolate_line_green(x1, y1, x2, y2, y_limit=200):
    slope = (y2 - y1) / (x2 - x1) if x2 != x1 else 0
    intercept = y1 - slope * x1

    y_bottom = y_limit
    x_bottom = int((y_bottom - intercept) / slope) if slope != 0 else int(x1)

    x_modified=(720-intercept)/slope

    return (x_modified, 720, x_bottom, y_bottom)

In [6]:
def extrapolate_line_red(x1, y1, x2, y2, y_limit=200):
    slope = (y2 - y1) / (x2 - x1) if x2 != x1 else 0
    intercept = y1 - slope * x1

    y_bottom = y_limit
    x_bottom = int((y_bottom - intercept) / slope) if slope != 0 else int(x1)

    x_modified=(500-intercept)/slope

    return (x_modified, 500, x_bottom, y_bottom)

In [7]:
def display_lines(image, left_lines, right_lines, y_limit):
    line_image = np.zeros_like(image)

    if left_lines:
        x1, y1, x2, y2 = average_lines(left_lines)
        if x1 is not None:
            x1, y1, x2, y2 = extrapolate_line_green(x1, y1, x2, y2, y_limit)
            cv2.line(line_image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 10)

    if right_lines:
        x1, y1, x2, y2 = average_lines(right_lines)
        if x1 is not None:
            x1, y1, x2, y2 = extrapolate_line_red(x1, y1, x2, y2, 720)
            cv2.line(line_image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 10)

    return line_image

## Lane Detection in Video

In [8]:
video_path = 'input_video.mp4'
cap = cv2.VideoCapture(video_path)

while(cap.isOpened()):
    ret, frame = cap.read()
    
    if not ret:
        break
    
    processed_image = process_image(frame)
    processed_image = (processed_image * 255).astype(np.uint8)
    
    edges = region_of_interest(processed_image)
    lines = cv2.HoughLinesP(edges, 2, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=50)
    if lines is not None:
        left_lines, right_lines = classify_lines_by_slope(lines)
        line_image = display_lines(frame, left_lines, right_lines, y_limit=500)
        combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
        cv2.imshow('Lane Detection', combo_image)
    else:
        break
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()